----------
# Introduction
__________

In this tutorial we will use pmx and Gromacs to set up alchemical free energy calculations for a relative free energy calculation study. We will further perform the simulations and estimate the actual ddG value.

The protein and ligand topologies for this case have already been generated and described in the study:<br>
*Gapsys, Perez-Benito et al. Large scale relative protein ligand binding affinities using non-equilibrium alchemy.  2020. Chemical Science<br>
https://pubs.rsc.org/en/content/articlelanding/2020/SC/C9SC03754C* <br>
All the input/output files from this work can be found on the pmx github page:<br>
*https://github.com/deGrootLab/pmx/tree/master/protLig_benchmark*

In the tutorial, we will use a pair of ligands binding to the JNK1 kinase and estimate the relative free energy difference $\Delta\Delta G$ between these molecules. For that, the following thermodynamic cycle will be used:

FIGURE: thermodynamic cycle


------------------
# Method
------------------
methodology of noneq 

---------------
Workflow
-----------------
The overall workflow consists of the following steps:<br>
0. Initialize the working environment
1. Prepare hybrid structures/topologies
2. Prepare equilibrium simulations
3. Prepare non-equilibrium transitions
4. Analysis

----------------
Step 0. Initialize working environment
-----------------

Here, we will define the paths to the protein and ligand topologies, as well as to the molecular dynamics parameter files (.mdp).

We will also set some main parameters: water model, salt concentration, number of simulation replicas to use.

In [5]:
import pmx
from pmx.utils import create_folder
from pmx import gmx, ligand_alchemy, jobscript
import sys
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 
%pylab inline
from IPython.core.display import clear_output
import os,shutil
import re
import subprocess
import glob
import random
from AZtutorial import *

Populating the interactive namespace from numpy and matplotlib


/home/vgapsys/anaconda2/envs/AZtutorial/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
# initialize the free energy environment object: it will store the main parameters for the calculations
fe = AZtutorial( )

# set the workpath
fe.workPath = 'workpath'
# set the path to the molecular dynamics parameter files
fe.mdpPath = 'input/mdppath'
# set the number of replicas (several repetitions of calculation are useful to obtain reliable statistics)
fe.replicas = 3
# provide the path to the protein structure and topology
fe.proteinPath = 'input/protein_amber'
# provide the path to the folder with ligand structures and topologies
fe.ligandPath = 'input/ligands'
# provide edges
fe.edges = [ ['18625-1','18626-1'] ]

# finally, let's prepare the overall free energy calculation directory structure
fe.prepareFreeEnergyDir( )

edge_18625-1_18626-1

---------------------
Summary of the setup:
---------------------

   workpath: /home/vgapsys/project/make_hybrid/AZtutorial/workpath
   mdp path: /home/vgapsys/project/make_hybrid/AZtutorial/input/mdppath
   protein files: /home/vgapsys/project/make_hybrid/AZtutorial/input/protein_amber
   ligand files: /home/vgapsys/project/make_hybrid/AZtutorial/input/ligands
   number of replicase: 3
   edges:
        edge_18625-1_18626-1

---------------------
Directory structure:
---------------------

/home/vgapsys/project/make_hybrid/AZtutorial/workpath/
|
|--edge_X_Y
|--|--water
|--|--|--stateA
|--|--|--|--run1/2/3
|--|--|--|--|--em/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2/3
|--|--|--|--|--em/eq_posre/eq/transitions
|--|--protein
|--|--|--stateA
|--|--|--|--run1/2/3
|--|--|--|--|--em/eq_posre/eq/transitions
|--|--|--stateB
|--|--|--|--run1/2/3
|--|--|--|--|--em/eq_posre/eq/transitions
|--|--hybridStrTop
|--edge_..
DONE


`1a. Firstly, let's create a hybrid structure/topology for the two ligands forming an edge.
For that we will establish a mapping of atoms to be morphed between the molecules.`

In [394]:
# this command will map the atoms of all edges found in the 'fe' object
# bVerbose flag prints the output of the command
fe.atom_mapping(bVerbose=False)

-----------------------
Performing atom mapping
-----------------------
edge_18625-1_18626-1
DONE


`1b. Secondly, we will construct a hybrid structure and topology based on the established mapping`

In [395]:
fe.hybrid_structure_topology(bVerbose=False)

----------------------------------
Creating hybrid structure/topology
----------------------------------
edge_18625-1_18626-1
DONE


`1c. Finally, we assemble the ligand and ligand+protein systems, i.e. create structures and topologies that will be used further in the step 2.`

In [396]:
fe.assemble_systems( )

----------------------
Assembling the systems
----------------------
edge_18625-1_18626-1
DONE


------------------
Step 2. Prepare equilibrium simulations.
---------------------
Prepare simulation boxes with solvent and ions.
Subsequently energy minimize the systems, prepare equilibrium simulations and start the runs.

`2a.` Build boxes, solvate, add ions

In [397]:
fe.boxWaterIons( )

----------------
Box, water, ions
----------------
edge_18625-1_18626-1
DONE


`2b.` Energy minimization.<br>
Prepare the simulation and subsequently either<br>
 a) run locally (suitable for the tutorial purpose)<br>
 b) create jobscripts to submit to the cluster

In [404]:
fe.prepare_simulation( simType='em' )

-----------------------------------------
Preparing simulation: em
-----------------------------------------
edge_18625-1_18626-1
DONE


Running energy minimization locally (e.g. on a laptop) will take 10-15 minutes for one edge.<br>
You can use the command below for that.
If you do not want to wait that long and have an access to a cluster, skip this step and proceed to the next step.

In [267]:
fe.run_simulation_locally( simType='em', bProt=False, bVerbose=True )

We can also submit the jobs to run on the cluster.<br>
This, naturally, requires some knowledge of the cluster available, queueing system and some dependencies. Several jobscript parameters allow to define the environment on the cluster. If there are additional cluster specifics, the jobscripts can be easily modified.<br>

Together with the jobscripts, a script `submit.py` will be generated. This script can be used to simply submit all the jobs to the cluster.

In [415]:
# set several parameters
fe.JOBqueue = 'SGE'
fe.JOBsource = ['/etc/profile.d/modules.sh','/usr/local/gromacs/GMXRC2018']
fe.JOBmodules = ['shared','owl/intel-mpi-default','cuda91']
fe.JOBgpu = True
fe.JOBgmx = 'mdrun_threads'

# create the jobscripts
fe.prepare_jobscripts(simType='em')

---------------------------------------------
Preparing jobscripts for: em
---------------------------------------------


`2c.` Once the energy minimization has finished, we can prepare equilibrium simulations.

In [413]:
fe.prepare_simulation( simType='eq' )

-----------------------------------------
Preparing simulation: eq
-----------------------------------------
edge_18625-1_18626-1
DONE


Again, prepare and submit the equilibrium simulation jobs to the cluster.<br>
This time the calculation will take longer (we are performing 6 ns simulations).<br>
On a node with a GPU and 8 CPUs simulations of ligand in water will take ~2 hours, ligand-protein complex will take ~10 hours.

In [423]:
fe.prepare_jobscripts(simType='eq')

---------------------------------------------
Preparing jobscripts for: eq
---------------------------------------------


--------------
Step 3. Prepare non-equilibrium transitions
---------------
At this point we already have the trajectories for equilibrium ensembles of two ligand states in water and in complex with the protein.<br>

`3a.` From these simulations we will extract the snapshots and initiate rapid transitions from one state to the other.

In [ ]:
fe.prepare_transitions( bGenTpr=True )

`3b.` Prepare submission scripts for transitions.<br>
Each transition is very short (50 ps), therefore, we can reduce the number of required CPUs and flood our cluster with many small jobs.

In [40]:
fe.JOBsimcpu = 2
fe.JOBqueue = 'SGE'
fe.JOBsource = ['/etc/profile.d/modules.sh','/usr/local/gromacs/GMXRC2018']
fe.JOBmodules = ['shared','owl/intel-mpi-default','cuda91']
fe.JOBgpu = True
fe.JOBgmx = 'mdrun_threads'

fe.prepare_jobscripts(simType='transitions', bLig=False)

---------------------------------------------
Preparing jobscripts for: transitions
---------------------------------------------
DONE


----------------
Step 4. Analysis
------------------

`4a.` Firstly, let's process the calculated work values and estimate free energy differences.<br>
The `pmx analyse` program will integrate the dhdl.xvg files to obtain the forward and reverse work values.<br>
Subsequently, the Crooks fluctuation theorem and Jarzynski estimators will be applied to relate the work distributions to $\Delta G$

In [ ]:
fe.run_analysis( bVerbose=True)

`4b.` Finally, we can collect all the results into one data structure and investigate the outcome.<br>
Here, we read the $\Delta G$ values and calculate $\Delta\Delta G$.<br>
The calculated error reflects the statistical uncertainty and the variation between multiple calculation repeats.